### Задание 1

In [1]:
from string import punctuation
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize

In [2]:
foyer = open('foyer.txt', encoding='cp1251').read()

In [3]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [4]:
def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [5]:
sntncs_foyer = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(foyer)]

In [6]:
unigrams_foyer = Counter()
for st in sntncs_foyer:
    unigrams_foyer.update(st)

In [7]:
bigrams_foyer = Counter()
for st in sntncs_foyer:
    bigrams_foyer.update(ngrammer(st))

In [8]:
trigrams_foyer = Counter()
for st in sntncs_foyer:
    trigrams_foyer.update(ngrammer(st, n=3))

In [9]:
id2word_uni = list(unigrams_foyer)
word2id_uni = {word:i for i, word in enumerate(id2word_uni)}

In [10]:
id2word_bi = list(bigrams_foyer)
word2id_bi = {word:i for i, word in enumerate(id2word_bi)}

In [11]:
matrix = np.zeros((len(bigrams_foyer), 
                   len(unigrams_foyer)))
for ngram in trigrams_foyer:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix[word2id_bi[bigram]][word2id_uni[word3]] =  (trigrams_foyer[ngram]/
                                                                     bigrams_foyer[bigram])

In [12]:
matrix_foyer_bi = np.zeros((len(unigrams_foyer), 
                   len(unigrams_foyer)))

for ngram in bigrams_foyer:
    word1, word2 = ngram.split()
    matrix_foyer_bi[word2id_uni[word1]][word2id_uni[word2]] =  (bigrams_foyer[ngram]/
                                                                     unigrams_foyer[word1])

In [13]:
def start_step(text, matrix_2, word2id_bi, id2word, start_idx):
    second_word = np.random.choice(matrix_2.shape[1], p=matrix_2[start_idx])
    word = id2word[second_word]
    text.append(word)
    current_bigram = "<start> {}".format(word)
    current_idx = word2id_bi[current_bigram]
    return current_bigram, current_idx

def generate(matrix, matrix_2, id2word, word2id_bi, word2id_uni, n=100):
    text = []
    start_idx = word2id_uni["<start>"]
    current_bigram, current_idx = start_step(text, matrix_2, word2id_bi, id2word, start_idx)
    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        word = id2word[chosen]
        
        if word == '<end>':
            current_bigram, current_idx = start_step(text, matrix_2, word2id_bi, id2word, start_idx)
        else:
            text.append(word)
            current_bigram = "{} {}".format(current_bigram.split()[1], word)
            current_idx = word2id_bi[current_bigram]
    
    return ' '.join(text)

In [14]:
print(generate(matrix, matrix_foyer_bi, id2word_uni, word2id_bi, word2id_uni, n=100).replace('<end>', '\n'))

почему не делал скульптур в америке всего пару месяцев ее тело нашли в рок крик парке который в версале который рядом с телефоном это конечно ничего не угрожало я гадала сказала ли она со своим другом и если бы целовалась сама он сказал «если заглянуть в словарь на слово «запредельная красота» там будет ваше фото» я сказал да потому что с папой иногда играли по воскресеньям называлась «разведывательная экспедиция» находить по выходным людей с фамилией блэк и выяснять у них зубы всю жизнь растут и если у того кто в «скорой» распознал прибор того кто внутри не распознал приборы тех кто


текст, сгенерированный биграммной моделью:

неуспевшие пока я с высокомерною улыбкой 
 нет послезавтра две бутылки вместо чистых социалистов 
 — но все-таки не ответив на всю жизнь так по комнате рассчитаетесь пожалуйте — вздохнул 
 но… но но хоть чаю и еще летом — ради как бы подхватили и маврикия николаевича 
 прибыли 
 ставрогин 
 ведь шатов 
 наступило затишье и опять револьвер петр верховенский тотчас же всё это зажило 
 я не мог иметь надо о его именьице оставшееся вино забиралось в петербург 
 я ведь еще «преданы фанатически» 
 фр 
 — наскучит не находит лампадку а 
 — к ней


Сравнивая новый текст со сгенерированными биграммной моделью, можно заметить, что в нашем случае он стал чуть более связным и состоящим из более осмысленных сегментов, которые в свою очередь стали крупнее. В новом тексте можно даже выделить какие-то отдельные предложения, например «почему не делал скульптур в америке», «ее тело нашли в рок крик парке который в версале», «он сказал «если заглянуть в словарь на слово «запредельная красота» там будет ваше фото»». В тексте биграммной модели «осмысленные» куски встречаются, но они не имеют никакого разграничения и как бы накладываются друг на друга.

### Задание 2

In [1]:
import re
import collections
from nltk import sent_tokenize
from nltk import word_tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import Text8Corpus
from gensim.test.utils import datapath

In [2]:
text = '''
В Новомосковском суде рассматривается уголовное дело о поставке наркотиков в тюрьму, которую, предположительно, организовал один из заключённых. Как полагает следствие, помощником, переносившим вещества на себе, был кот — он родился и вырос в колонии. Животное якобы пользовалось небольшим лазом в тюремном ограждении — на «воле» его поймал поставщик, надел на него ошейник с тайным кармашком, в который и положил свёрток с веществами. Обвинение утверждает, что есть даже трое свидетелей, которые могут опознать кота.

Животное, конечно, не проходило подозреваемым по делу, однако было признано вещественным доказательством — его поместили в специальный зооуголок на хранение. Когда защита решила провести следственный эксперимент с котом, выяснилось, что он сбежал ещё зимой. «Нам разрешили выпускать кота из клетки, потому что ему было холодно. Мы его выпускали. Но как-то раз к нам забежали чужие собаки, а собак он нереально боялся и убежал», — рассказывает сотрудница зооуголка.

Теперь защита считает, что дело стоит закрыть — ведь доказать, что животное могло переносить наркотики, уже невозможно. Более того, адвокат подсудимого не доверяет версии следствия — по его мнению, «администрация колонии не могла допустить наличия дыр в ограждении, следовательно, кота можно было только перебросить». Собственно, на этом и базировался следственный эксперимент: «Надеть на кота ошейник с грузом, равным весу наркотического вещества, и посмотреть, как кот будет себя с ним вести, привычно ли это для него. Затем кота нужно перебросить через забор с колючей проволокой и посмотреть, проследует ли он к месту назначения в колонии», — говорит адвокат.

Однако шанс доказать факт преступления всё ещё есть: например, если ранее кота осмотрели и нашли на нём следы наркотиков.'''

In [3]:
sntncs_2 = sent_tokenize(text)

In [4]:
sntncs_2

['\nВ Новомосковском суде рассматривается уголовное дело о поставке наркотиков в тюрьму, которую, предположительно, организовал один из заключённых.',
 'Как полагает следствие, помощником, переносившим вещества на себе, был кот — он родился и вырос в колонии.',
 'Животное якобы пользовалось небольшим лазом в тюремном ограждении — на «воле» его поймал поставщик, надел на него ошейник с тайным кармашком, в который и положил свёрток с веществами.',
 'Обвинение утверждает, что есть даже трое свидетелей, которые могут опознать кота.',
 'Животное, конечно, не проходило подозреваемым по делу, однако было признано вещественным доказательством — его поместили в специальный зооуголок на хранение.',
 'Когда защита решила провести следственный эксперимент с котом, выяснилось, что он сбежал ещё зимой.',
 '«Нам разрешили выпускать кота из клетки, потому что ему было холодно.',
 'Мы его выпускали.',
 'Но как-то раз к нам забежали чужие собаки, а собак он нереально боялся и убежал», — рассказывает сот

In [5]:
def spl(word): 
    return [char for char in word]  

In [6]:
voc = []
for i in range(len(sntncs_2)):
    voc.append(spl(sntncs_2[i]))

In [7]:
print(voc)

[['\n', 'В', ' ', 'Н', 'о', 'в', 'о', 'м', 'о', 'с', 'к', 'о', 'в', 'с', 'к', 'о', 'м', ' ', 'с', 'у', 'д', 'е', ' ', 'р', 'а', 'с', 'с', 'м', 'а', 'т', 'р', 'и', 'в', 'а', 'е', 'т', 'с', 'я', ' ', 'у', 'г', 'о', 'л', 'о', 'в', 'н', 'о', 'е', ' ', 'д', 'е', 'л', 'о', ' ', 'о', ' ', 'п', 'о', 'с', 'т', 'а', 'в', 'к', 'е', ' ', 'н', 'а', 'р', 'к', 'о', 'т', 'и', 'к', 'о', 'в', ' ', 'в', ' ', 'т', 'ю', 'р', 'ь', 'м', 'у', ',', ' ', 'к', 'о', 'т', 'о', 'р', 'у', 'ю', ',', ' ', 'п', 'р', 'е', 'д', 'п', 'о', 'л', 'о', 'ж', 'и', 'т', 'е', 'л', 'ь', 'н', 'о', ',', ' ', 'о', 'р', 'г', 'а', 'н', 'и', 'з', 'о', 'в', 'а', 'л', ' ', 'о', 'д', 'и', 'н', ' ', 'и', 'з', ' ', 'з', 'а', 'к', 'л', 'ю', 'ч', 'ё', 'н', 'н', 'ы', 'х', '.'], ['К', 'а', 'к', ' ', 'п', 'о', 'л', 'а', 'г', 'а', 'е', 'т', ' ', 'с', 'л', 'е', 'д', 'с', 'т', 'в', 'и', 'е', ',', ' ', 'п', 'о', 'м', 'о', 'щ', 'н', 'и', 'к', 'о', 'м', ',', ' ', 'п', 'е', 'р', 'е', 'н', 'о', 'с', 'и', 'в', 'ш', 'и', 'м', ' ', 'в', 'е', 'щ', 'е', 'с', 

In [72]:
phrases = Phrases(voc, min_count=0.00000001, scoring='npmi', threshold=-1)

In [76]:
phrases = Phrases(voc, min_count=1, scoring='npmi', threshold=-1)
phrases_input = phrases[voc]
for i in range(100):
    phrases = Phrases(phrases_input, min_count=1, scoring='npmi', threshold=-1)
    phrases_input = phrases[phrases_input]

In [79]:
print(list(phrases[phrases_input]))

[['\n_В_ _Н_о_в_о_м_о_с_к_о_в_с_к_о_м_ _с_у_д_е_ _р_а_с_с_м_а_т_р_и_в_а_е_т_с_я_ _у_г_о_л_о_в_н_о_е_ _д_е_л_о_ _о_ _п_о_с_т_а_в_к_е_ _н_а_р_к_о_т_и_к_о_в_ _в_ _т_ю_р_ь_м_у_,_ _к_о_т_о_р_у_ю_,_ _п_р_е_д_п_о_л_о_ж_и_т_е_л_ь_н_о_,_ _о_р_г_а_н_и_з_о_в_а_л_ _о_д_и_н_ _и_з_ _з_а_к_л_ю_ч_ё_н_н_ы_х_.'], ['К_а_к_ _п_о_л_а_г_а_е_т_ _с_л_е_д_с_т_в_и_е_,_ _п_о_м_о_щ_н_и_к_о_м_,_ _п_е_р_е_н_о_с_и_в_ш_и_м_ _в_е_щ_е_с_т_в_а_ _н_а_ _с_е_б_е_,_ _б_ы_л_ _к_о_т_ _—_ _о_н_ _р_о_д_и_л_с_я_ _и_ _в_ы_р_о_с_ _в_ _к_о_л_о_н_и_и_.'], ['Ж_и_в_о_т_н_о_е_ _я_к_о_б_ы_ _п_о_л_ь_з_о_в_а_л_о_с_ь_ _н_е_б_о_л_ь_ш_и_м_ _л_а_з_о_м_ _в_ _т_ю_р_е_м_н_о_м_ _о_г_р_а_ж_д_е_н_и_и_ _—_ _н_а_ _«_в_о_л_е_»_ _е_г_о_ _п_о_й_м_а_л_ _п_о_с_т_а_в_щ_и_к_,_ _н_а_д_е_л_ _н_а_ _н_е_г_о_ _о_ш_е_й_н_и_к_ _с_ _т_а_й_н_ы_м_ _к_а_р_м_а_ш_к_о_м_,_ _в_ _к_о_т_о_р_ы_й_ _и_ _п_о_л_о_ж_и_л_ _с_в_ё_р_т_о_к_ _с_ _в_е_щ_е_с_т_в_а_м_и_.'], ['О_б_в_и_н_е_н_и_е_ _у_т_в_е_р_ж_д_а_е_т_,_ _ч_т_о_ _е_с_т_ь_ _д_а_ж_е_ _т_р_о_е_ _с_в_и_д_е_т_е_л_е_й_,_ _к_о_т_о